In [1]:
import sys

sys.argv = [""]
del sys

import warnings
warnings.filterwarnings('ignore')

In [2]:
import argparse
import os

import numpy as np
import torch
from torch.utils.data import DataLoader

from inpaint.data import PlacesDataset 

In [3]:
def flipCoin():
    f = random.random()
    return True if f < 0.5 else False

In [4]:
def create_cfg(args):
    parser = argparse.ArgumentParser()
    for key, val in args.items():
        parser.add_argument(key, default=val)

    cfg = parser.parse_args()
    return cfg

In [5]:
args = {}

# dataset config
args["--val_ds_dir"] = '../../../../Datasets/Places365_Standard/val_large'
args["--crop_size"] = (256, 256)
args["--batch_size"] =  1
args["--num_workers"] = 1

In [6]:
cfg = create_cfg(args)

In [7]:
val_ds = PlacesDataset(path_dir = cfg.val_ds_dir, transform_config=('to_tensor', 'center_crop', 'norm'), crop_size=cfg.crop_size)
val_ds.imglist = val_ds.imglist[:5000]
print(f"Total validation images: {len(val_ds)}")

Total validation images: 5000


In [8]:
val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=True, pin_memory=True, num_workers=cfg.num_workers)

___

In [9]:
from inpaint.core.generator import GatedGenerator
from inpaint.tools import Evaluate
import glob

___

1. Initialize generator from a saved checkpoint

In [10]:
args = {}

# model | generator config
args["--in_channels"] = 4
args["--out_channels"] = 3
args["--latent_channels"] = 64
args["--pad_type"] = "zero"
args["--activation"] = "elu"
args["--norm_d"] = "none"
args["--norm_g"] = "batch"
args["--init_type"] = "kaiming"
args["--init_gain"] = 0.02
args["--use_cuda"] = True
args["--use_perceptualnet"] = True
args["--sn_enable"] = True
args["--add_context_attention"] = True

In [11]:
cfg = create_cfg(args)

In [12]:
generator = GatedGenerator(cfg)

ckpt_path = "../../../experiments/inpaint/ckpts/exp11/best_models_final.pth"

models = torch.load(ckpt_path, map_location=torch.device('cpu'))
gen_model_state_dict = models["generator_state_dict"]

generator.load_state_dict(gen_model_state_dict)

<All keys matched successfully>

___

2. Create Evaluate instance pass dataloader and generator instance to Evaluate 

In [13]:
eval_gen = Evaluate(generator,val_loader)

____
3. Evaluate (return avg psnr and avg ssim value

In [14]:
avg_psnr, avg_ssim = eval_gen.evaluate()
avg_psnr, avg_ssim

(70.65689195655214, 0.9059241479516029)